In [1]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Загрузим данные

In [3]:
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NLP/test_data.csv')

In [4]:
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [5]:
data.shape

(480, 4)

In [6]:
data.dlg_id.nunique()

6

У нас есть 6 диалогов

**a. Извлечение реплик с приветствием — где менеджер поздоровался.**

In [6]:
#создаём список возможных приветствий
hello = re.compile('привет|здравствуйте|добрый день|добрый вечер|день добрый')

In [ ]:
#формируем список реплик с приветствием и добавим соответствующий столбец в data
greet_phrase = {}
data['greeting_manager'] = np.zeros(480, dtype=bool)
for el in range(data.shape[0]):
  if data.role.iloc[el] == 'manager':
    if re.search(hello, data.text.iloc[el].lower()) != None:
      data['greeting_manager'].iloc[el] = True
      greet_phrase[data.dlg_id.iloc[el]] =  data.text.iloc[el]

In [18]:
greet_phrase

{0: 'Алло здравствуйте',
 1: 'Алло здравствуйте',
 2: 'Алло здравствуйте',
 3: 'Алло дмитрий добрый день'}

У нас в списке есть 4 фразы. То есть в 4 из 6 диалогах менеджер поздоровался

**b. Извлечение реплик, где менеджер представил себя.** 

In [19]:
#функция с регулярным выражением
import re
def get_greet_and_name(text):
  call = re.compile('меня зовут|зовут')
  return re.search(call, text.lower()) != None

In [20]:
#получаем список нужных фраз
phrases_with_greetings = {}

for el in range(data.shape[0]):
  if data.role.iloc[el] == 'manager':  
    if get_greet_and_name(data.text.iloc[el]) == True:
      phrases_with_greetings[data.dlg_id.iloc[el]] = data.text.iloc[el]

In [21]:
phrases_with_greetings

{0: 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
 1: 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
 2: 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
 3: 'Добрый меня максим зовут компания китобизнес удобно говорить'}

**c. Извлечение имени менеджера**

In [22]:
#будем искать по словарю фраз, где менеджер представил себя
#подгрузим базу с именами
import json

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/russian_names.json', 'r', encoding='utf-8-sig') as f:
    names = json.load(f)

only_names = []
for i in range(len(names)):
  only_names.append(names[i]['Name'].lower())

In [25]:
#токенизируем реплики для удобства и узнаем части речи

def preprocess(phrase):
    phrase = nltk.word_tokenize(phrase, language="russian")
    phrase = nltk.pos_tag(phrase, lang='rus')
    return phrase

In [26]:
manager_names = {}

for k, v in phrases_with_greetings.items():
  phrase = preprocess(v.lower())
  for word in phrase:
    if word[-1] == 'S':
      if word[0] in only_names:
        manager_names[k] = word[0]

In [27]:
manager_names

{0: 'ангелина', 1: 'ангелина', 2: 'ангелина', 3: 'максим'}

**d. Извлечение названия компании**

In [28]:
#будем искать по словарю фраз, где менеджер представился
companies_names = {}

for k, v in phrases_with_greetings.items():
  phrase = preprocess(v.lower())
  start = 0
  end = 0
  for i in range(len(phrase)):
    if phrase[i][0] == 'компания': 
      start = i+1
      counter = 0
      for j in phrase[start:]:
        counter += 1
        if j[1] == 'S':
          end = start + counter
          break
      name_of_company = []
      for w in phrase[start:end]:
        name_of_company.append(w[0])
      companies_names[k] = ' '.join(name_of_company)

In [29]:
companies_names

{0: 'диджитал бизнес',
 1: 'диджитал бизнес',
 2: 'диджитал бизнес',
 3: 'китобизнес'}

**e. Извлечение реплик, где менеджер попрощался**

In [35]:
#создаём список возможных слов для завершения диалога
bye = re.compile('до свидания|всего доброго|до встречи')

In [ ]:
#формируем список реплик с прощанием и добавим соответствующий столбец в data
bye_phrase = {}
data['bye_manager'] = np.zeros(480, dtype=bool)

for el in range(data.shape[0]):
  if data.role.iloc[el] == 'manager':
    if re.search(bye, data.text.iloc[el].lower()) != None:
      data['bye_manager'].iloc[el] = True
      bye_phrase[data.dlg_id.iloc[el]] = data.text.iloc[el]

In [37]:
bye_phrase

{0: 'Всего хорошего до свидания',
 1: 'До свидания',
 3: 'Угу все хорошо да понедельника тогда всего доброго',
 4: 'Во вторник все ну с вами да тогда до вторника до свидания',
 5: 'Ну до свидания хорошего вечера'}

Менеджеры попрощались в пяти диалогах из шести

**f. Проверка требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»**

In [44]:
#создадим список номеров диалогов, где требование было выполнено
correct_dialog = []
for i in greet_phrase:
    for j in bye_phrase:
        if i == j:
            correct_dialog.append(i)
            break

correct_dialog

[0, 1, 3]